# Data Preprocessing: ERA5, Flow, and Basin Outlines

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import os
import pickle
from netCDF4 import Dataset
from shapely.geometry import MultiPolygon, Polygon
from pyproj import Proj, transform

In [ ]:
def get_betweenDates(initDate,finDate):
    
    """
    out: years, months, days, dayInds -- np arrays of the dates between (inclusive) initDate and finDate
    in: initDate: [yyyy,mm,dd]
        finDate: [yyyy,mm,dd]
    
    example:
    
    import numpy as np
    initDate = [1979,1,1]
    finDate = [2010,12,31]
    years,months,days,dayInds = get_betweenDates(initDate,finDate)
    """

    monthsInYear = np.hstack([1*np.ones((1,31)), 2*np.ones((1,28)), 3*np.ones((1,31)), 4*np.ones((1,30)), 5*np.ones((1,31)),
                              6*np.ones((1,30)), 7*np.ones((1,31)), 8*np.ones((1,31)), 9*np.ones((1,30)), 10*np.ones((1,31)), 
                              11*np.ones((1,30)), 12*np.ones((1,31))])
    monthsInYear = monthsInYear[0]
    monthsInYear_ly = np.hstack([1*np.ones((1,31)), 2*np.ones((1,29)), 3*np.ones((1,31)), 4*np.ones((1,30)), 5*np.ones((1,31)), 
                                 6*np.ones((1,30)), 7*np.ones((1,31)), 8*np.ones((1,31)), 9*np.ones((1,30)), 10*np.ones((1,31)), 
                                 11*np.ones((1,30)), 12*np.ones((1,31))])
    monthsInYear_ly = monthsInYear_ly[0]
    daysInYear = np.hstack([range(1,32), range(1,29), range(1,32), range(1,31), range(1,32), range(1,31), range(1,32), 
                  range(1,32), range(1,31), range(1,32), range(1,31), range(1,32)])
    daysInYear_ly = np.hstack([range(1,32), range(1,30), range(1,32), range(1,31), range(1,32), range(1,31), range(1,32), 
                  range(1,32), range(1,31), range(1,32), range(1,31), range(1,32)])

    years = []
    months = []
    days = []
    dayInds = []
    for year in range(initDate[0],finDate[0]+1): #for each year, append on the right day/month/year vector

        if np.mod(year,4)!=0: #if it is not a leap year
            years.append(year*np.ones((1,365)))
            months.append(monthsInYear)
            days.append(daysInYear)
            dayInds.append(range(1,366))
        else: #if it is a leap year
            years.append(year*np.ones((1,366)))
            months.append(monthsInYear_ly)
            days.append(daysInYear_ly)
            dayInds.append(range(1,367))

    years = np.hstack(years)[0]
    months = np.hstack(months)
    days = np.hstack(days)
    dayInds = np.hstack(dayInds)
    
    return years, months, days, dayInds

# Temperature and Precipitation

In [ ]:
def get_ProvinceDailyTemp(saveVars=0):

    """
    Compute daily temperature fields from ERA 5 dataset

    example:
    import geopandas as gpd
    import numpy as np
    from netCDF4 import Dataset
    tempDict = get_ProvinceDailyTemp(saveVars=0)
    """

    #open netcdf file
    fileDirERA = './Data/ERA5/'

    filenameERA = 'ERA5_T_1979_2015_6hourly_075_grid_AB_BC.nc'
    filePathERA = fileDirERA + filenameERA
    ERA = Dataset(filePathERA)

    #extract data from file
    lonERA = ERA.variables['longitude'][:] #longitude in degrees W 
    latERA = ERA.variables['latitude'][:]
    hoursERA = ERA.variables['time'][:] #hours since Jan 1, 1900
    T_hourly = ERA.variables['t2m'][:] #2-metre temperature, in Kelvin

    #convert temp data at 00:00, 06:00, 12:00, 18:00 to daily averages
    timestamps_per_day = 4
    T = []
    Tmax = []
    Tmin = []
    for daynum in range(int(len(hoursERA)/timestamps_per_day)): #for each day
        inds = list(range(daynum*timestamps_per_day,daynum*timestamps_per_day+timestamps_per_day))
        T.append(np.mean(T_hourly[inds,:,:],axis=0))
        Tmax.append(np.max(T_hourly[inds,:,:],axis=0))
        Tmin.append(np.min(T_hourly[inds,:,:],axis=0))

    initDate = [1979,1,1]
    finDate = [2015,12,31]
    years,months,days,dayInds = get_betweenDates(initDate,finDate)

    tempDict = {
        'T':T,
        'Tmax':Tmax,
        'Tmin':Tmin,
        'lonERA':lonERA,
        'latERA':latERA,
        'hoursERA':hoursERA,
        'yearsERA':years,
        'monthsERA':months,
        'daysERA':days        
    }

    if saveVars:

        pickle_filename = 'tempDict_ERA5_' + str(initDate[0]) + '_' + str(finDate[0]) + '_075grid_AB_BC.pickle'
        pickle_out = open(pickle_filename,'wb')
        pickle.dump(tempDict,pickle_out)
        pickle_out.close()

    return tempDict

In [ ]:
def get_ProvinceDailyPrec(saveVars=0):

    """
    Compute daily total precipitation fields from ERA 5 dataset
    
    example:
    import geopandas as gpd
    import numpy as np
    from netCDF4 import Dataset
    precDict = get_ProvinceDailyPrec(saveVars=0)
    """
    
    #open netcdf file
    fileDirERA = './Data/ERA5/'
        
    filenameERA = 'ERA5_P_1979_2015_6hourly_075_grid_AB_BC.nc'
    filePathERA = fileDirERA + filenameERA
    ERA = Dataset(filePathERA)

    #extract data from file
    lonERA = ERA.variables['longitude'][:] #longitude in degrees W 
    latERA = ERA.variables['latitude'][:]
    hoursERA = ERA.variables['time'][:] #hours since Jan 1, 1900
    P_6hourly = ERA.variables['tp'][:] #accumulated precipitation, in mm

    #convert data at 00:00, 06:00, 12:00, 18:00 to daily totals
    timestamps_per_day = 4
    P = []
    for daynum in range(int(len(hoursERA)/timestamps_per_day)): #for each day
        inds = list(range(daynum*timestamps_per_day,daynum*timestamps_per_day+timestamps_per_day))
        P.append(np.sum(P_6hourly[inds,:,:],axis=0))
        
    initDate = [1979,1,1]
    finDate = [2015,12,31]
    years,months,days,dayInds = get_betweenDates(initDate,finDate)
        
    precDict = {
        'P':P,
        'lonERA':lonERA,
        'latERA':latERA,
        'hoursERA':hoursERA,
        'yearsERA':years,
        'monthsERA':months,
        'daysERA':days        
    }
        
    if saveVars:
        pickle_filename = 'precDict_ERA5_' + str(initDate[0]) + '_' + str(finDate[0]) + '_075grid_AB_BC.pickle'
        pickle_out = open(pickle_filename,'wb')
        pickle.dump(precDict,pickle_out)
        pickle_out.close()   

    return precDict

In [ ]:
tempDict = get_ProvinceDailyTemp(saveVars = 0)

In [ ]:
precDict = get_ProvinceDailyPrec(saveVars = 0)

In [ ]:
tempDict = get_ProvinceDailyTemp(saveVars = 0)

In [ ]:
T = np.asarray(tempDict['T'])
Tmax = np.asarray(tempDict['Tmax'])
Tmin = np.asarray(tempDict['Tmin'])
P = np.asarray(precDict['P'])

eraLon = tempDict['lonERA']
eraLat = tempDict['latERA']
extentERA = [np.min(eraLon), np.max(eraLon), np.min(eraLat), np.max(eraLat)]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(np.mean(T,axis = 0), aspect = 'auto', cmap = 'RdBu', extent = extentERA)
ax.set_title('Mean Temperature')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(np.mean(P,axis = 0), aspect = 'auto', cmap = 'RdBu', extent = extentERA)
ax.set_title('Mean Precipitation')
plt.show()

# Flow

In [ ]:
def get_ProvinceFlow(prov, yearRange, frac_missing_total_max, frac_missing_yearly_max, n_worse_years, computeFlow, saveFlowVars, params, verbose):

    initDate = datetime(yearRange[0],1,1)
    finDate = datetime(yearRange[1],12,31)
    Nyears = len(range(yearRange[0], yearRange[1]+1))

    folderWithFiles = './Data/Flow/' +prov #folder that contains all of the flow files

    #get names of all files in folderWithFiles
    flowFiles = []
    for root, dirs, files in os.walk(folderWithFiles):
        for name in sorted(files):
            if name[0]=='0' or name[0]=='1': #if the current file is one that contains flow data (all flow files have a 0 or 1 at the start)
                flowFiles.append(os.path.join(root,name)) #full path name of this file
            else:
                if name[0] != '.': #if it is not .DS_store, ie: if it is the table of data describing all files
                    infofilename = os.path.join(root,name)

        df = pd.read_csv(infofilename, encoding = 'ISO-8859-1') #read in summary data of all streamflow files
        df.columns = [col.strip() for col in df.columns] #some columns have write space in the name -- remove this
        df = df.drop([0]) #remove empty row
        df.index = range(len(df)) #redo column indices (otherwise will start at 1 since we removed row 0)
        totalStations = len(df) #the total number of stations present, including those that have missing data and/or wrong years
        goodStations = [] #a list of indices of the stations which have sufficient data and the years of interest

    dfs_flow_good = []

    if computeFlow:

    #####first, check to see if each station has enough data and the right years

        yearmin = np.zeros([totalStations,1]) #minimum year with data in each station
        yearmax = np.zeros_like(yearmin) #maximum year with data in each station

        frac_missing_total_all = []
        frac_missing_yearly_all = []

        for station in range(totalStations): #for all stations (good and bad) that are active + naturalized flow in Alberta
            
            if verbose:
                print('Checking station: ', station, '/', totalStations-1)

            filename = flowFiles[station] #filename of current station
            df_flow = pd.read_csv(filename) #create dataframe of all flow data for this station
            df_flow = df_flow.drop([len(df_flow)-2,len(df_flow)-1]) #remove last two rows: not data, this is a disclaimer in each file

            dates = df_flow['Date'] #these are the dates that are in the data -- missing dates are omitted and we will need to fill those eventually
            dates_dt = [datetime.strptime(date,'%Y/%m/%d') for date in dates] #dates in datetime format for easier use
            df_flow['Datetime'] = dates_dt #create datetime column in flow dataframe

            df_flow.index = dates_dt #turn index from integers to datetime; can then fill missing data easily
            df_flow = df_flow.reindex(pd.date_range(initDate,finDate),fill_value=np.nan) #missing dates are filled with nan; clip to be dates within yearRange

            #calculate total fraction of missing days
            frac_missing_total = np.sum(np.isnan(df_flow['Flow'])) / len(df_flow['Flow'])

            #calculate fraction of missing days each year
            frac_missing_yearly = [] #empty list -- append with fraction of missing dates in each year at this station
            flow_mat = np.empty((Nyears, 365)) #rows are years, columns are days of that year (will ignore 366th day in leap years)
            for year in range(Nyears): #for each year in yearRange
                startDT = datetime(yearRange[0]+year, 1, 1) #starting date is January 1 of year
                finDT = startDT + timedelta(days = 364) #ending date is 364 days later
                flow_mat[year,:] = df_flow['Flow'][startDT:finDT] #row in flow_mat is the flow in this year
                frac_missing_this_year = np.sum(np.isnan(flow_mat[year,:])) / 365 #number of missing dates this year divided by 365
                frac_missing_yearly.append(frac_missing_this_year) #append fraction of missing dates

            #consider it a good station if the total fraction of missing data is acceptable AND the missing fraction each year is acceptable
            if frac_missing_total <= frac_missing_total_max and np.sort(frac_missing_yearly)[- n_worse_years - 1] <= frac_missing_yearly_max: #if the total fraction and yearly 
                goodStations.append(station)
                dfs_flow_good.append(df_flow)

            frac_missing_total_all.append(frac_missing_total)
            frac_missing_yearly_all.append(frac_missing_yearly)

    #####now, for each good station, compute flow/etc

        N = len(goodStations) #number of good stations
        all_flowseason = np.zeros([N,365])
        all_flowseason_NF = np.zeros_like(all_flowseason)
        all_flowseason_norm = np.zeros_like(all_flowseason)
        all_flowseason_norm_NF = np.zeros_like(all_flowseason)
        all_flowseason_norm_smooth = np.zeros_like(all_flowseason)
        all_flowseason_norm_smooth_NF = np.zeros_like(all_flowseason)
        yearvec = []
        all_flow = []
        all_flow_NF = []
        all_flowwindow = []
        all_flowwindow_NF = []
        all_flowwindow_norm = []
        all_flowwindow_norm_NF = []

        for ind, station in enumerate(goodStations):
            
            if verbose:
                print('Calculating good station: ', ind, '/', len(goodStations)-1)

            df_flow = dfs_flow_good[ind]
            flow_mat = np.empty((Nyears, 365))

            for year in range(Nyears):
                startDT = datetime(yearRange[0]+year, 1, 1)
                finDT = startDT + timedelta(days = 364)
                flow_mat[year,:] = df_flow['Flow'][startDT:finDT]

            all_flowseason[ind] = np.squeeze(np.nanmean(flow_mat, axis = 0))
            all_flowseason_NF[ind] = np.copy(all_flowseason[ind])
            all_flowseason_NF[ind][np.isnan(all_flowseason[ind])] = np.nanmin(all_flowseason[ind])

            flowseason_mean = np.mean(all_flowseason_NF[ind])
            flowseason_std = np.std(all_flowseason_NF[ind])

            all_flowseason_norm[ind] = (all_flowseason[ind] - flowseason_mean) / flowseason_std
            all_flowseason_norm_NF[ind] = (all_flowseason_NF[ind] - flowseason_mean) / flowseason_std

            all_flow.append(df_flow['Flow'])

            flow_mat_NF = np.copy(flow_mat)
            #flow_NF = np.empty((1,1))
            for year in range(Nyears):
                nan_inds = np.argwhere(np.isnan(flow_mat_NF[year,:]))
                flow_mat_NF[year, nan_inds] = all_flowseason_NF[ind][nan_inds]
                if year == 0:
                    flow_NF = np.expand_dims(flow_mat_NF[year,:], axis = 1)
                else:
                    flow_NF = np.vstack((flow_NF, np.expand_dims(flow_mat_NF[year,:], axis = 1)))
                if np.mod(yearRange[0] + year, 4)==0: #if leap year
                    flow_NF = np.vstack((flow_NF, flow_NF[-1]))

            all_flow_NF.append(flow_NF)

    stationID = df['Station'].iloc[goodStations]
    stationName = df['StationName'].iloc[goodStations]
    stationLat = df['Latitude'].astype(float).iloc[goodStations]
    stationLon = df['Longitude'].astype(float).iloc[goodStations]
    stationDrainageArea = df['DrainageArea'].iloc[goodStations]

    windowDatesTimestamp = pd.date_range(initDate,finDate)  
    windowDates = [datetime.strftime(ii,'%Y-%m-%d') for ii in windowDatesTimestamp]
    windowYears = np.asarray([int(d[0:4]) for d in windowDates])
    windowMonths = np.asarray([int(d[5:7]) for d in windowDates])
    windowDays = np.asarray([int(d[8:10]) for d in windowDates])

    flowDict = {
        'stationID':stationID,
        'stationName':stationName,
        'stationLat':stationLat,
        'stationLon':stationLon,
        'stationDrainageArea':stationDrainageArea,
        'all_flowseason':all_flowseason,
        'all_flowseason_NF':all_flowseason_NF,
        'all_flowseason_norm':all_flowseason_norm,
        'all_flowseason_norm_NF':all_flowseason_norm_NF,
        #'all_flowseason_norm_smooth':all_flowseason_norm_smooth,
        'all_flow':all_flow,
        'all_flow_NF':all_flow_NF,
        #'all_flowwindow':all_flowwindow,
        #'all_flowwindow_NF':all_flowwindow_NF,
        #'all_flowwindow_norm':all_flowwindow_norm,
        #'all_flowwindow_norm_NF':all_flowwindow_norm_NF,
        'windowDates':windowDates,
        'windowYears':windowYears,
        'windowMonths':windowMonths,
        'windowDays':windowDays,
        'params' : params
    }

    if saveFlowVars:
        pickle_out = open(prov + '_flowvars_' + str(yearRange[0]) +  '_' + str(yearRange[1]) + '_missing_' + str(int(100*frac_missing_total_max)) + '_' + str(int(100*frac_missing_yearly_max)) + '_' + str(n_worse_years) + '.pickle','wb')
        pickle.dump(flowDict,pickle_out)
        pickle_out.close()
        
    return flowDict

In [ ]:
yearRange = [1979, 2015]
saveFlowVars = 0
frac_missing_total_max = 0.4
frac_missing_yearly_max = 0.4
n_worse_years = 1 #number of years that can have greater than the 'frac_missing_yearly_max' fraction of missing data
computeFlow = 1
verbose = 0

params = {'prov' : 'AB',
          'yearRange' : yearRange,
         'frac_missing_total_max' : frac_missing_total_max,
         'frac_missing_yearly_max' : frac_missing_yearly_max,
         'n_worse_years' : n_worse_years}

flowAB = get_ProvinceFlow('AB', 
                          yearRange, 
                          frac_missing_total_max, 
                          frac_missing_yearly_max, 
                          n_worse_years, 
                          computeFlow, 
                          saveFlowVars, 
                          params,
                          verbose)

params = {'prov' : 'BC',
          'yearRange' : yearRange,
         'frac_missing_total_max' : frac_missing_total_max,
         'frac_missing_yearly_max' : frac_missing_yearly_max,
         'n_worse_years' : n_worse_years}

flowBC = get_ProvinceFlow('BC', 
                          yearRange, 
                          frac_missing_total_max, 
                          frac_missing_yearly_max, 
                          n_worse_years, 
                          computeFlow, 
                          saveFlowVars, 
                          params, 
                          verbose)

In [ ]:
all_flow_AB = np.asarray(flowAB['all_flow']) #all flow data from AB stations
all_flow_NF_AB = np.asarray(flowAB['all_flow_NF']) #all flow data from AB stations, with nans filled (NF)
all_flow_BC = np.asarray(flowBC['all_flow']) #all flow data from BC stations
all_flow_NF_BC = np.asarray(flowBC['all_flow_NF']) #all flow data from BC stations, with nans filled (NF)

fig, ax = plt.subplots(nrows = 1, ncols = 1)
ind = 0
ax.plot(all_flow_NF_AB[ind][:365],'k--', linewidth = 1, label = 'NaN Filled')
ax.plot(all_flow_AB[ind][:365], label = 'Original')
ax.legend()
ax.set_xlabel('Day of Year')
ax.set_ylabel('Streamflow [$m^3/s$]')

plt.show()

# Basin Outlines

In [ ]:
#first, need to load flow data and extract stationIDs to match with the Water Survey of Canada basin outlines

dir_data = './Data'

flowpickle = ['BC_flowvars_1979_2015_missing_40_40_1.pickle', 'AB_flowvars_1979_2015_missing_40_40_1.pickle'] #filenames of .pickle files which contain AB/BC streamflow data

#open flow data; since flow data is provided at provincial level, loop through/open/concatenate data from desired provinces
flowDicts = []
for flowfile in flowpickle:
  pickle_in = open(dir_data + '/Flow/' + flowfile,'rb')
  flowDicts.append(pickle.load(pickle_in))

#store flow data as a dictionary
flowDict = {
    'stationID' : np.hstack((flowDicts[0]['stationID'],flowDicts[1]['stationID'])), #station ID numbers
    'stationLat' : np.hstack((flowDicts[0]['stationLat'],flowDicts[1]['stationLat'])), #latitude of each station, in degrees
    'stationLon' : np.hstack((flowDicts[0]['stationLon'],flowDicts[1]['stationLon'])), #longitude of each station, in degrees
}

#unpack data
stationLat = flowDict['stationLat']
stationLon = flowDict['stationLon']
stationID = flowDict['stationID']

In [ ]:
#create list of unique filename prefixes in the WSC folder (the same filename prefix will have .gdbtable, .gdptablx, .spx, and .gdbindexes)

dir_basins = './Data/WSC_Basins.gdb/'

polygon_files = []
unique_filenames = []

for root, dirs, files in os.walk(dir_basins):
    for name in sorted(files):
        file = os.path.join(root,name)
        file_short = file.replace('/','.').split('.')[-2]
        polygon_files.append(file) #full path name of this file
        if file_short[0] == 'a':
            unique_filenames.append(file.replace('/','.').split('.')[-2])

unique_filenames = list(set(unique_filenames))

In [ ]:
#there can be issues in identifying the CRS when reading the basin data using gpd.read_file
#we will manually assign the CRS and define a transformation to lat/lon
#the basin outline data is Canada_Albers_Equal_Area_Conic (ESRI:102001)
#we will project it to latitude/longitude (EPSG:4326)

inProj = Proj(init='esri:102001')
outProj = Proj(init='epsg:4326')

#these can be used to transform points x,y in Canada_Albers_Equal_Area_Conic to x1, y1 in lat/lon by: x1,y1 = transform(inProj, outProj, x, y)

In [ ]:
stationBasins = [None] * len(stationID)

for zz, filename in enumerate(unique_filenames): #for each unique file prefix

    #print statement every 100th file for keeping track
    if np.mod(zz,100) == 0:
        print(str(zz+1) + '/' + str(len(unique_filenames)))

    #load one unique file
    basin = gpd.read_file(root + filename + '.gdbtable') #read in basin info
    
    if len(basin.columns) == 8: #for properly formatted basins, there should be 8 columns (at least one basin is improperly formatted)

        if basin.columns[0] != 'Station': #sometimes the first column is 'Station', sometimes 'STATION'; change all to 'Station' for indexing
            curr_name = basin.columns[0] #'Station' or 'STATION'
            basin = basin.rename(columns = {curr_name : 'Station'}) #change to 'Station'

        if basin['Station'][0] in stationID: #if this current basin is a station in flow data

            #reproject into lat/lon and overwrite the old projection polygon
            x,y = basin['geometry'][0][0].exterior.xy
            x1,y1 = transform(inProj, outProj, x, y)

            kk = np.argwhere(basin['Station'][0] == stationID)[0][0]
            stationBasins[kk] = Polygon([[x1[kk],y1[kk]] for kk in range(len(x1))]) #the basin of each stationID in flow data

        

In [ ]:
fig, ax = plt.subplots(figsize = (12,12))

plot_prov_ax(prov = ['BC','AB'], ax = ax) 

for basin in stationBasins:
    if basin is not None:
        x,y = basin.exterior.xy
        ax.plot(x, y, color = 'gray', zorder = 0, linewidth = 0.5)
        ax.scatter(stationLon, stationLat, edgecolor = 'k', facecolor = 'w', s = 75, linewidth = 1, zorder = 3)

plt.show()

In [ ]:
saveIt = 0

yearRange = [1979,2015]

if saveIt:
    pickle_out = open('WSC_basins_' + str(yearRange[0]) +  '_' + str(yearRange[1]) + '_missing_' + str(int(100*frac_missing_total_max)) + '_' + str(int(100*frac_missing_yearly_max)) + '_' + str(n_worse_years) + '.pickle','wb')
    pickle.dump(stationBasins,pickle_out)
    pickle_out.close()